In [ ]:
import requests
from anytree import Node, RenderTree
from collections import defaultdict
from time import sleep

In [ ]:
def get_technical_skills_from_vacancies():
    """Получаем технические навыки из описаний вакансий"""
    base_url = "https://api.hh.ru/vacancies"
    skills = set()
    
    params = {
        "text": "разработчик OR программист",
        "search_field": "name",
        "per_page": 50,
        "page": 0
    }
    
    try:
        for page in range(2):
            params["page"] = page
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if "items" not in data:
                break
                
            for item in data["items"]:
                if item.get('url'):
                    vacancy_response = requests.get(item['url'])
                    if vacancy_response.status_code == 200:
                        vacancy_data = vacancy_response.json()
                        if 'key_skills' in vacancy_data:
                            for skill in vacancy_data['key_skills']:
                                skills.add(skill['name'].lower())
                    sleep(0.5)
                    
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к API HH: {e}")
    
    return list(skills)

In [ ]:
def build_skills_tree(skills):
    """Строим дерево навыков на основе категорий с указанием источников"""
    categories = {
        "Языки программирования": {
            "Низкоуровневые": ["c/c++", "assembler", "rust", "go", "c++"],
            "Высокоуровневые": ["python", "java", "javascript", "c#", "php", "ruby", "typescript"],
            "Функциональные": ["haskell", "scala", "erlang", "clojure", "f#"],
            "Скриптовые": ["python", "javascript", "perl", "lua", "bash"]
        },
        "Базы данных": {
            "SQL": ["sql", "postgresql", "mysql", "oracle", "sql server", "postgres"],
            "NoSQL": ["mongodb", "redis", "cassandra", "elasticsearch", "dynamodb"]
        },
        "Фреймворки": {
            "Веб": ["django", "flask", "spring", "laravel", "ruby on rails", "express"],
            "Мобильные": ["react native", "flutter", "xamarin"],
            "Тестирование": ["pytest", "junit", "selenium", "jest", "mocha"]
        },
        "DevOps": {
            "Контейнеризация": ["docker", "kubernetes", "docker-compose"],
            "CI/CD": ["jenkins", "gitlab ci", "github actions", "teamcity"],
            "Облака": ["aws", "azure", "google cloud", "gcp", "amazon web services"]
        },
        "Frontend": {
            "Основы": ["html", "css", "javascript", "html5", "css3"],
            "Фреймворки": ["react", "angular", "vue.js", "svelte"]
        },
        "Backend": {
            "Серверные языки": ["java", "c#", "python", "php", "ruby", "node.js"],
            "API": ["rest", "graphql", "grpc", "soap"]
        }
    }
    
    # Создаем корневой узел
    root = Node("Технические навыки")
    
    # Словарь для связи навыков с категориями
    skill_sources = {}
    
    # Сначала заполняем словарь источников
    for category, subcategories in categories.items():
        for subcategory, subcategory_skills in subcategories.items():
            for skill in subcategory_skills:
                if skill not in skill_sources:
                    skill_sources[skill] = []
                skill_sources[skill].append(f"{category} → {subcategory}")
    
    # Строим дерево
    category_nodes = {}
    subcategory_nodes = defaultdict(dict)
    matched_skills = set()
    
    for category, subcategories in categories.items():
        category_node = Node(category, parent=root)
        category_nodes[category] = category_node
        
        for subcategory, subcategory_skills in subcategories.items():
            subcategory_node = Node(subcategory, parent=category_node)
            subcategory_nodes[category][subcategory] = subcategory_node
            
            for skill in subcategory_skills:
                if skill in skills:
                    matched_skills.add(skill)
                    Node(skill, parent=subcategory_node)
    
    # Обрабатываем навыки, которые не попали в категории
    other_skills = [s for s in skills if s not in matched_skills]
    if other_skills:
        other_node = Node("Другие навыки (не распределены)", parent=root)
        for skill in other_skills:
            # Пытаемся найти похожие навыки в наших категориях
            similar = []
            for defined_skill, sources in skill_sources.items():
                if defined_skill in skill or skill in defined_skill:
                    similar.extend(sources)
            
            if similar:
                # Если нашли похожие, добавляем информацию
                sources = ", ".join(set(similar))  # убираем дубли
                Node(f"{skill} (возможно из: {sources})", parent=other_node)
            else:
                # Если ничего не нашли
                Node(f"{skill} (категория не определена)", parent=other_node)
    
    return root

In [ ]:
print("Получаем технические навыки из вакансий HH...")
skills = get_technical_skills_from_vacancies()
print(f"Получено {len(skills)} уникальных навыков")

tree = build_skills_tree(skills)

print("\nДерево технических навыков:")
for pre, _, node in RenderTree(tree):
    print(f"{pre}{node.name}")